## Imports

In [7]:
import pickle
import matching_utils

{}


## Load detections

In [ ]:
# Open the file in binary mode
with open('/home/group05/mcv-m6-2022-team5/week5/sort_bbox_seq3_c13.pkl', 'rb') as file:
      
    # Call load method to deserialze
    myvar = pickle.load(file)
  
    print(myvar)